In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join('/home/yzwang/MF/github')))
sys.path.append(os.path.abspath(os.path.join('/home/yzwang/MF/github/Method')))
import numpy as np
import torch
import torch.nn as nn
from datetime import datetime
from utils import (root_mean_squared_error, mean_absolute_error, create_lagged_data, three_frequency_convert_to_lowest, plot_forecasts_multi)
from Baseline import set_seed
set_seed(42)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#data parameters
d1 = 20
d2 = 3
P1 = 13
P2 = 56
QP3 = 9
Q = 5
DD = d1*d2*P1+ d2*P2+ QP3

# Model architecture (LSTM with linear projection)
class LSTM1(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, output_dim=9):
        super().__init__()
        self.LSTM = nn.LSTM(input_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim) 
        self.dropout = nn.Dropout(0.3)
    
    def forward(self, x):
        LSTM_out, _ = self.LSTM(x)
        # LSTM_out = self.dropout(LSTM_out)
        return self.fc(LSTM_out)


Pre-COVID

In [2]:
T = 188
T_val = 20
T_test = 20
T_train = T - T_val - T_test

T_M = T * d2
T_H = T * d1 * d2
print('T',T,'T_H:', T_H, 'T_M:', T_M)
print('T_train:', T_train, 'T_val:', T_val, 'T_test:', T_test)

Y = np.load(f'../data_clean/Y_std.npy')[:,:T]
X_M = np.load(f'../data_clean/X_M_std.npy')[:,:T*d2]
X_H = np.load(f'../data_clean/X_H_std.npy')[:,:T*d1*d2]

print('Y shape:', Y.shape)
print('X_M shape:', X_M.shape)
print('X_H shape:', X_H.shape)
print('Y max:', np.max(Y), 'Y min:', np.min(Y), 'Y mean:', np.mean(Y), 'Y std:', np.std(Y))
print('X_M max:', np.max(X_M), 'X_M min:', np.min(X_M), 'X_M mean:', np.mean(X_M), 'X_M std:', np.std(X_M))
print('X_H max:', np.max(X_H), 'X_H min:', np.min(X_H), 'X_H mean:', np.mean(X_H), 'X_H std:', np.std(X_H))


T 188 T_H: 11280 T_M: 564
T_train: 148 T_val: 20 T_test: 20
Y shape: (9, 188)
X_M shape: (56, 564)
X_H shape: (13, 11280)
Y max: 9.083060657379114 Y min: -4.5941919722762705 Y mean: 0.003900497296983884 Y std: 0.87206867900193
X_M max: 16.940348022306395 X_M min: -17.12817591268496 X_M mean: 0.00039118324278265517 X_M std: 0.8661968246394718
X_H max: 23.195031218753527 X_H min: -25.564933640672024 X_H mean: 7.850425864522082e-05 X_H std: 0.9699726059297447


In [3]:
EPOCHS = 100
LEARNING_RATE = 0.001
Hidden_dim = 128
k = 1
Y_total = three_frequency_convert_to_lowest(Y, X_M, X_H, d1, d2)
print('Y_total shape:', Y_total.shape)
model = LSTM1(input_dim=DD*k, hidden_dim=Hidden_dim, output_dim=Q).to(device)
model = LSTM1(input_dim=DD*k, hidden_dim=Hidden_dim, output_dim=Q).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

forecast_test_LSTM = np.zeros((Q, T_test)) 
for t in range(T_test):
    print(f'==== Step {t}')
    X_train_cur = Y_total[:,:T_train + T_val + t -1]
    X_train_lag_cur = create_lagged_data(X_train_cur, k)  #(P3*k, T_train + T_val+t-k) #(10,43)
    # print('X_L_train_lagged_cur', X_train_lag_cur.shape)  
    Y_train_cur = Y_total[:Q, k:T_train + T_val + t]  #(P3, T_train + T_val+t-k) #(5,43)
    # print('Y_train_cur', Y_train_cur.shape)
    # Training phase
    model.train()
    for epoch in range(EPOCHS):
        optimizer.zero_grad()
        y_pred = model(torch.FloatTensor(X_train_lag_cur.T).to(device)).T
        loss = criterion(y_pred, torch.FloatTensor(Y_train_cur).to(device))
        loss.backward()
        optimizer.step()
    X_test_cur = Y_total[:, T_train + T_val + t - k:T_train + T_val + t].reshape(-1, 1)
    model.eval()
    with torch.no_grad():
        y_pred = model(torch.FloatTensor(X_test_cur.T).unsqueeze(0).to(device)).reshape(-1)  #(P3)
    print('X_test_cur.T',X_test_cur.T[0,:Q])
    forecast_test_LSTM[:, t] = y_pred.cpu().numpy()

Y_test = Y_total[:Q, T_train + T_val:]
err_MAFE_t_LSTM, err_MAFE_LSTM = mean_absolute_error(Y_test, forecast_test_LSTM)
err_t_LSTM, err_LSTM = root_mean_squared_error(Y_test, forecast_test_LSTM)
print("\n=== LSTM Results ===")
print("True matrix:", Y_test.T)
print('forecast matrix:', np.round(forecast_test_LSTM.T,4))
print("\nMAFE:", err_MAFE_t_LSTM)
print("RMSFE:", err_t_LSTM)
print("Mean MAFE:", err_MAFE_LSTM)
print("Mean RMSFE:", err_LSTM) 

Y_total shape: (957, 188)
==== Step 0
X_test_cur.T [ 0.49526466  0.24280682  0.3556868   0.29394212 -0.75624627]
==== Step 1
X_test_cur.T [-0.15164106  0.39048279 -0.13241863 -0.60718242 -1.00196544]
==== Step 2
X_test_cur.T [ 0.20876589  0.04326467  0.64202131  0.20936176 -0.47583844]
==== Step 3
X_test_cur.T [-0.04771408 -0.02812461 -0.11317301  0.65873711  2.17582818]
==== Step 4
X_test_cur.T [-0.24819097 -0.01382807 -0.15790623  0.11723854 -0.95281688]
==== Step 5
X_test_cur.T [-4.45991064e-01 -2.83034902e-01 -3.61565128e-01 -8.34441362e-04
 -8.46476528e-01]
==== Step 6
X_test_cur.T [-0.08416703  0.03213965 -0.31940928  0.58131048 -0.21149163]
==== Step 7
X_test_cur.T [-0.32067452 -0.20210718 -0.31845437 -0.61050166  2.47013531]
==== Step 8
X_test_cur.T [ 0.03447353 -0.02148607 -0.10559435  0.00580862 -1.4169627 ]
==== Step 9
X_test_cur.T [-0.10722422 -0.18434579  0.46407312 -0.30548092  0.63848285]
==== Step 10
X_test_cur.T [-0.16887996  0.0447052  -0.27861508 -0.85229339  0.19258

Post_COVID

In [4]:
T = 208
T_val = 20
T_test = 20
T_train = T - T_val - T_test

T_M = T * d2
T_H = T * d1 * d2
print('T',T,'T_H:', T_H, 'T_M:', T_M)
print('T_train:', T_train, 'T_val:', T_val, 'T_test:', T_test)

Y = np.load(f'../data_clean/Y_std.npy')[:,:T]
X_M = np.load(f'../data_clean/X_M_std.npy')[:,:T*d2]
X_H = np.load(f'../data_clean/X_H_std.npy')[:,:T*d1*d2]

print('Y shape:', Y.shape)
print('X_M shape:', X_M.shape)
print('X_H shape:', X_H.shape)
print('Y max:', np.max(Y), 'Y min:', np.min(Y), 'Y mean:', np.mean(Y), 'Y std:', np.std(Y))
print('X_M max:', np.max(X_M), 'X_M min:', np.min(X_M), 'X_M mean:', np.mean(X_M), 'X_M std:', np.std(X_M))
print('X_H max:', np.max(X_H), 'X_H min:', np.min(X_H), 'X_H mean:', np.mean(X_H), 'X_H std:', np.std(X_H))


T 208 T_H: 12480 T_M: 624
T_train: 168 T_val: 20 T_test: 20
Y shape: (9, 208)
X_M shape: (56, 624)
X_H shape: (13, 12480)
Y max: 9.083060657379114 Y min: -8.981419418585231 Y mean: -7.591268544445516e-18 Y std: 1.0
X_M max: 24.694614539572694 X_M min: -24.295703738737302 X_M mean: 3.100897642039128e-18 X_M std: 1.0
X_H max: 30.816445727888585 X_H min: -33.0391984692243 X_H mean: -1.25255930983351e-17 X_H std: 1.0000000000000169


In [5]:
EPOCHS = 100
LEARNING_RATE = 0.001
Hidden_dim = 128
k = 1
Y = three_frequency_convert_to_lowest(Y, X_M, X_H, d1, d2)
print('Y shape:', Y.shape)
model = LSTM1(input_dim=DD*k, hidden_dim=Hidden_dim, output_dim=Q).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

forecast_test_LSTM = np.zeros((Q, T_test)) 
for t in range(T_test):
    print(f'==== Step {t}')
    X_train_cur = Y[:,:T_train + T_val + t -1]
    X_train_lag_cur = create_lagged_data(X_train_cur, k)  #(P3*k, T_train + T_val+t-k) #(10,43)
    # print('X_L_train_lagged_cur', X_train_lag_cur.shape)  
    Y_train_cur = Y[:Q, k:T_train + T_val + t]  #(P3, T_train + T_val+t-k) #(5,43)
    # print('Y_train_cur', Y_train_cur.shape)
    # Training phase
    model.train()
    for epoch in range(EPOCHS):
        optimizer.zero_grad()
        y_pred = model(torch.FloatTensor(X_train_lag_cur.T).to(device)).T
        loss = criterion(y_pred, torch.FloatTensor(Y_train_cur).to(device))
        loss.backward()
        optimizer.step()
    X_test_cur = Y[:, T_train + T_val + t - k:T_train + T_val + t].reshape(-1, 1)
    model.eval()
    with torch.no_grad():
        y_pred = model(torch.FloatTensor(X_test_cur.T).unsqueeze(0).to(device)).reshape(-1)  #(P3)
    forecast_test_LSTM[:, t] = y_pred.cpu().numpy()

Y_test = Y[:Q, T_train + T_val:]
err_MAFE_t_LSTM, err_MAFE_LSTM = mean_absolute_error(Y_test, forecast_test_LSTM)
err_t_LSTM, err_LSTM = root_mean_squared_error(Y_test, forecast_test_LSTM)
print("\n=== LSTM Results ===")
print("True matrix:", Y_test.T)
print('forecast matrix:', np.round(forecast_test_LSTM.T,4))
print("\nMAFE:", err_MAFE_t_LSTM)
print("RMSFE:", err_t_LSTM)
print("Mean MAFE:", err_MAFE_LSTM)
print("Mean RMSFE:", err_LSTM) 

Y shape: (957, 208)
==== Step 0
==== Step 1
==== Step 2
==== Step 3
==== Step 4
==== Step 5
==== Step 6
==== Step 7
==== Step 8
==== Step 9
==== Step 10
==== Step 11
==== Step 12
==== Step 13
==== Step 14
==== Step 15
==== Step 16
==== Step 17
==== Step 18
==== Step 19

=== LSTM Results ===
True matrix: [[ 9.35299674e-03 -5.73500062e-02 -5.37466604e-01  4.53523986e-01
  -6.90598153e-02]
 [-1.90467483e+00 -2.20816306e+00 -8.59470962e-01  7.86579180e-01
  -8.61880784e-02]
 [-8.19074112e+00 -8.98141942e+00 -3.94762545e+00  2.05825258e+00
  -3.36392389e+00]
 [ 6.31581678e+00  7.19688935e+00  4.02044130e+00 -2.39049351e+00
   6.01634051e+00]
 [ 3.76494454e-01  6.23911998e-01  5.49274968e-01 -9.81139316e-01
  -1.24411281e+00]
 [ 6.44838192e-01  1.40706160e+00 -3.64888751e-01  1.02050197e+00
   2.31995032e+00]
 [ 8.16859588e-01  2.35007443e+00 -6.21447463e-01 -1.81818875e+00
   6.18844696e-01]
 [ 1.65349660e-01  5.30525958e-02  7.30768160e-01 -9.84265177e-01
  -3.97239167e-01]
 [ 1.02763318e+